In [1]:
import pandas as pd

In [3]:
traindf = pd.read_csv("train.csv")
testdf = pd.read_csv("test.csv")

In [4]:
traindf.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
print(traindf.text[0])

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) 
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. 
As we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emails 

In [6]:
#train data size 

print(f"shape of train data: {traindf.shape}")
print(f"shape of test  data: {testdf.shape}")


shape of train data: (20800, 5)
shape of test  data: (5200, 4)


In [7]:
#check for missing value in dataframe 

traindf.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
#removing missing value df 
traindf.dropna(how='any' , inplace= True)

In [9]:
#recheck shape of train data 
traindf.shape

(18285, 5)

In [10]:
#droping prediction variable 
#train df with out label
traindf_woLabel = traindf.drop('label' , axis = 1)

In [11]:
traindf.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [12]:
traindf_woLabel.columns

Index(['id', 'title', 'author', 'text'], dtype='object')

In [13]:
#generator function to fecth single row of df 
def fnGenrator(df):
    for row in df.itertuples():
        yield row    

In [14]:
#preprocessing for stop words removal
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
nlp = spacy.load("en_core_web_lg")

In [15]:
def preprocess(text , NLP = nlp):
    doc = nlp(text)    
    return [token.lemma_ for token in doc if (token.is_stop == False and token.is_alpha == True)]

#rename review to newsArticle 
reviews = np.zeros((len(traindf_woLabel), 1)) 
vectorizer = TfidfVectorizer(tokenizer = preprocess)
reviews= vectorizer.fit_transform(traindf_woLabel['text'])

In [16]:
reviews

<18285x133544 sparse matrix of type '<class 'numpy.float64'>'
	with 4393782 stored elements in Compressed Sparse Row format>

In [17]:
testdf.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [18]:
submit = pd.read_csv("submit.csv")
submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [19]:
testdf.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [20]:
#test vectorizer
newsArticleTest = pd.merge(testdf , submit , on = ['id'])

In [21]:
newsArticleTest.isnull().sum()

id          0
title     122
author    503
text        7
label       0
dtype: int64

In [22]:
#filter for na 
newsArticleTest.dropna(how='any' , inplace= True)

In [23]:
newsArticleTest.head()

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,0
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",1
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori...",1


In [24]:
testVector = vectorizer.fit_transform(newsArticleTest['text'])

In [25]:
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [26]:
#dummyclassifier 
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(reviews , traindf["label"])

DummyClassifier(strategy='most_frequent')

In [27]:
dummyPred = dummy_clf.predict(testVector)
print(classification_report(newsArticleTest['label'] , dummyPred))

              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2213
           1       0.00      0.00      0.00      2362

    accuracy                           0.48      4575
   macro avg       0.24      0.50      0.33      4575
weighted avg       0.23      0.48      0.32      4575



C:\Users\aaman\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
import pickle



In [2]:
print(datetime.now())

2021-03-13 14:25:05.943359
